In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.optimizer_v2 import adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [ ]:
image_path_affectnet = "./AffectNet/images.npy"
emotion_path_affectnet = "./AffectNet/emotions.npy"

images_affectnet = np.load(image_path_affectnet)

#images_affectnet = images_affectnet/127.5 - 1

images_affectnet = tf.convert_to_tensor(images_affectnet)

images_affectnet = layers.Rescaling(1./127.5, offset=-1)(images_affectnet)

emotions_affectnet = np.load(emotion_path_affectnet)
emotions_affectnet = tf.convert_to_tensor(emotions_affectnet)

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
def train(model, learning_rate, loss, num_epochs, batch_size, save_path):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    model.fit(x=images_affectnet,
              y=emotions_affectnet,
              batch_size=batch_size,
              epochs=num_epochs)
    model.save(save_path)
    # model = tf.keras.models.load_model(save_path)
    del model
    gc.collect()

In [ ]:
def create_model(base_model):
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./saved_models/'):
    os.mkdir('./saved_models/')
learning_rate = 1e-4
num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()
for i in range(2):
    if i == 0: # AffectNet
        base_model = vgg16.VGG16(include_top=False, weights=None, input_shape=(48,48,3))
        model = create_model(base_model)
        save_path = './saved_models/none_affectnet'
        train(model, learning_rate, loss, num_epochs, batch_size, save_path)
    if i == 1: # ImageNet-AffectNet
        base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(48,48,3))
        model = create_model(base_model)
        save_path = './saved_models/imagenet_affectnet'
        train(model, learning_rate, loss, num_epochs, batch_size, save_path)